Nama : Najla Dhia Rusydi

NIM : 164221043

Nama : Najla Dhia Rusydi

NIM : 164221043

# Import Library

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity


# Import data

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

df_obat = pd.read_csv('data_halodoc.csv')
df_eval = pd.read_csv('dataset pertanyaan.csv')

df_obat

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,name,price,Aturan Pakai,Dosis,Efek Samping,Golongan Produk,Indikasi Umum,Kemasan,Komposisi,Kontra Indikasi,Manufaktur,No. Registrasi,Perhatian,uses
0,Sildenafil Citrate 50 mg 4 Tablet,Rp96.500 - Rp168.500,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet",Sildenafil citrate 50 mg,Hipersensitivitas. Pasien yang menggunakan nit...,Generic Manufacturer,BPOM: GKL1433531717A1* *) Obat ini merupakan o...,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,SILDENAFIL CITRATE merupakan obat yang digunak...
1,Viagra 50 mg 1 Tablet,Rp156.800 - Rp164.900,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Blister @ 4 Tablet Salut Selaput",Sildenafil citrate 50 mg,Pasien yang menggunakan nitrat organik intermi...,Pfizer,BPOM: DKI1690401417A1,HARUS DENGAN RESEP DOKTER. Pasien pria yang ti...,VIAGRA merupakan obat yang digunakan untuk men...
2,Ericfil Tablet 100 mg,Rp77.100 - Rp104.000,Dikonsumsi satu jam sebelum melakukan aktivita...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, muka merah, dispepsia, hidung te...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 1 Strip @ 4 Tablet salut selaput",Sildenafil citrate 100 mg,"Pasien yang menggunakan nitrat organik, baik s...",Novell Pharmaceutical Laboratories,BPOM: DKL1333528917A1,HARUS DENGAN RESEP DOKTER. Pria yang tidak dia...,ERICFIL merupakan obat yang digunakan untuk me...
3,Cialis Tablet 10 mg,Rp197.300 - Rp203.500,Diberikan sebelum atau setelah makan,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, pusing, dispepsia, nyeri punggun...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 2 Blister @ 2 Tablet Salut Selaput",Tadalafil 10 mg,"Pemberian bersama dengan nitrat oksida, nitrat...",NaN,NaN,NaN,CIALIS 10 MG merupakan obat dengan kandungan T...
4,Ericfil 50 mg Odf,Rp36.500 - Rp44.400,Cara penggunaan Ericfil ODF: 1. Buka kemasan d...,PENGGUNAAN OBAT INI HARUS SESUAI DENGAN PETUNJ...,"Sakit kepala, rasa panas dan kemerahan pada ku...",Obat Keras (Merah),INFORMASI OBAT INI HANYA UNTUK KALANGAN MEDIS....,"Dus, 4 Strip @ 1 ODF",Sildenafil Citrate 50 mg,Hipersensitif terhadap komponen VIASTAR BLUE 5...,Novell Pharmaceutical Laboratories,BPOM: DKL2233554519A1,Hati-hati penggunaan obat in pada pasien denga...,ERICFIL 50 MG ODF merupakan obat yang digunaka...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4727,Optimum Nutrition Whey Gold Standard Vanilla 1...,"Rp2,200,000",Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,Pangan Olahan Diet Khusus,"Membantu pembentukan otot, mempertahankan mass...",Pack @ 4.54 Kg,"Protein Blend (Whey Protein Isolate, Whey Prot...",NaN,Vita Shopindo,NaN,NaN,Optimum Nutrition On Whey Gold Standard Vanill...
4728,Optimum Nutrition Whey Gold Standard Vanilla 5 Lb,"Rp1,150,000",Dikonsumsi 30-60 menit seletah latihan dan ber...,Campurkan 1 scoop ON Whey dengan 1 gelas air (...,NaN,Pangan Olahan Diet Khusus,"Membantu pembentukan otot, mempertahankan mass...",Botol @ 2.27 Kg,"Protein Blend (Whey Protein Isolate, Whey Prot...",NaN,Vita Shopindo,NaN,NaN,Optimum Nutrition On Whey Gold Standard Vanill...
4729,Optimum Serious Mass Chocolate 12Lb,Rp1.100.000 - Rp1.100.100,Dikonsumsi diantara waktu makan atau kapanpun ...,Campurkan 2 scoop Serious Mass dengan 3 gelas ...,NaN,Pangan Olahan Diet Khusus,Membantu meningkatkan berat badan dengan kandu...,"Pack @ 5,44 kg","Maltodextrin, Protein Blend (Whey Protein Conc...",NaN,Radiant Sentral Nutrindo,BPOM: ML262331074648,NaN,Optimum Nutrition On Serious Mass Chocolate 12...
4730,Optimum Whey Gold Chocolate 10 Lb

# Data Preprocessing

In [3]:
def preprocess_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('indonesian'))
    tokens = [token for token in tokens if token not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

columns = df_obat.columns
df_obat['dokumen'] = df_obat.apply(lambda x: ' '.join([str(x[col]) if pd.notna(x[col]) else '' for col in columns]), axis=1)

df_obat['processed_text'] = df_obat['dokumen'].apply(preprocess_text)

df_obat[['name', 'processed_text']]


,name,processed_text
0,Sildenafil Citrate 50 mg 4 Tablet,sildenafil citrat 50 mg 4 tablet rp96500 rp168...
1,Viagra 50 mg 1 Tablet,viagra 50 mg 1 tablet rp156800 rp164900 dikons...
2,Ericfil Tablet 100 mg,ericfil tablet 100 mg rp77100 rp104000 dikonsu...
3,Cialis Tablet 10 mg,ciali tablet 10 mg rp197300 rp203500 makan pen...
4,Ericfil 50 mg Odf,ericfil 50 mg odf rp36500 rp44400 penggunaan e...
...,...,...
4727,Optimum Nutrition Whey Gold Standard Vanilla 1...,optimum nutrit whey gold standard vanilla 10 l...
4728,Optimum Nutrition Whey Gold Standard Vanilla 5 Lb,optimum nutrit whey gold standard vanilla 5 lb...
4729,Optimum Serious Mass Chocolate 12Lb,optimum seriou mass chocol 12lb rp1100000 rp11...
4730,Optimum Whey Gold Chocolate 10 Lb,optimum whey gold chocol 10 lb rp2200000 dikon...


In [4]:
df_eval ['processed_question'] = df_eval['Pertanyaan'].apply(preprocess_text)
df_eval

,Pertanyaan,processed_question
0,Comtusi obat apa?,comtusi obat
1,balsem murah?,balsem murah
2,obat pereda nyeri gigi?,obat pereda nyeri gigi
3,Mengapa Boost Optimum tidak cocok untuk orang ...,boost optimum cocok orang dewasa gangguan medi
4,Mengapa penting untuk menggunakan SGM BBLR ata...,sgm bblr rekomendasi dokter
5,Bagaimana Sensi Dry Pants menjaga kesehatan ku...,sensi dri pant menjaga kesehatan kulit bayi


# Method 1: TF-IDF dengan Cosine Similarity

cara kerja :
1. pertanyaan dan dokumen diubah mennjadi TF-IDF
2. menghitung cosine similiarity antara dokumen dan pertanyaan
3. diambil 3 teratas yang memiliki skor relevansi tertinggi

In [5]:
def tfidf_search(query, documents):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([query] + documents)
    cosine_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
    return cosine_scores[0]

all_results = []
for idx, row in df_eval.iterrows():
    query = row['processed_question']  
    documents = df_obat['processed_text'].to_list()  
    
    tfidf_scores = tfidf_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen' : documents,
        'TF-IDF Score': tfidf_scores
    })
    
    results = results.sort_values(by='TF-IDF Score', ascending=False).head(3)  
    results['Query'] = row['Pertanyaan']
    all_results.append(results)

final_results = pd.concat(all_results, ignore_index=True)
final_results

,Nama_Obat,Dokumen,TF-IDF Score,Query
0,Comtusi Forte 10 Kapsul,comtusi fort 10 kapsul rp29100 rp32000 makan p...,0.356923,Comtusi obat apa?
1,Comtusi Sirup 60 ml,comtusi sirup 60 ml rp43000 rp55000 perut koso...,0.285191,Comtusi obat apa?
2,Comtusi Rasa Strawberry Syrup 60 ml,comtusi strawberri syrup 60 ml rp43200 rp52300...,0.259508,Comtusi obat apa?
3,Copal Balsem 25 g,copal balsem 25 g rp17800 rp25900 oleskan memb...,0.340178,balsem murah?
4,Copal Balsem 36 g,copal balsem 36 g rp19500 rp28700 gosokkan tub...,0.317188,balsem murah?
5,Tresnojoyo Balsem Telon 40 g,tresnojoyo balsem telon 40 g rp16300 rp19700 h...,0.301671,balsem murah?
6,Pamol Sirup 60 ml,pamol sirup 60 ml rp20200 rp44000 makan anak 6...,0.322780,obat pereda nyeri gigi?
7,Pamol 500 mg 10 Tablet,pamol 500 mg 10 tablet rp9800 rp12700 dikonsum...,0.308762,obat pereda nyeri gigi?
8,Bimastan 500 mg 10 Kaplet,bimastan 500 mg 10 kaplet rp3900 rp6600 makan ...,0.300797,obat pereda nyeri gigi?
9,Boost Optimum 400 g,boost optimum 400 g rp118000 rp216000 larutkan...,0.417445,Mengapa Boost Optimum tidak cocok untuk orang ...


dilihat dari hasil yang di dapat metode ini dapat dengan baik mencari dokumen terkait.

- pertanyaan 1 : semua dokumen relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : 2 dokumen teratas memang merupakan obat nyeri namun bukan untuk gigi, sedangkan yang satunya memang tenetang gigi namun bukan untuk obt nyeri.
- pertanyaan 4 : semua dokumen relevant
- pertanyaan 5 : semua dokumen relevant
- pertanyaan 6 : semua dokumen relevant

# Method 2: BM25

cara kerja :
1. dukumen dan pertanyaan diubah menjadi TF-IDF
2. menghitung panjang dokumen dan rata-rata penjang dokumen
3. melakukan perhitungan skor BM25 untuk tiap-tiap dokumen, dengan mempertimbangkan frekuensi kata, IDF, panjang dokumen,dan parameter 𝑘1 dan b.
4. diambil 3 teratas dengan skor terbaik

In [6]:
def bm25_search(query, documents, k1=1.5, b=0.75):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([query])
    
    doc_lengths = np.sum(tfidf_matrix.toarray(), axis=1)
    avg_doc_length = np.mean(doc_lengths)
    
    tfidf_array = tfidf_matrix.toarray()
    query_array = query_vector.toarray()[0]
    
    scores = []
    for i in range(len(documents)):
        score = 0
        for term_idx in range(len(query_array)):
            if query_array[term_idx] > 0:
                tf = tfidf_array[i, term_idx]
                idf = vectorizer.idf_[term_idx]
                numerator = tf * (k1 + 1)
                denominator = tf + k1 * (1 - b + b * doc_lengths[i] / avg_doc_length)
                score += idf * numerator / denominator
        scores.append(score)
    
    return np.array(scores)

all_results = []
for idx, row in df_eval.iterrows():
    query = row['processed_question'] 
    documents = df_obat['processed_text'].tolist()  
    
    bm25_scores = bm25_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen' : documents,
        'BM25 Score': bm25_scores
    })
    
    results = results.sort_values(by='BM25 Score', ascending=False).head(3)  
    results['Query'] = row['Pertanyaan']  
    all_results.append(results)

final_results = pd.concat(all_results, ignore_index=True)

final_results


,Nama_Obat,Dokumen,BM25 Score,Query
0,Comtusi Forte 10 Kapsul,comtusi fort 10 kapsul rp29100 rp32000 makan p...,3.398481,Comtusi obat apa?
1,Comtusi Sirup 60 ml,comtusi sirup 60 ml rp43000 rp55000 perut koso...,2.828736,Comtusi obat apa?
2,Comtusi Rasa Strawberry Syrup 60 ml,comtusi strawberri syrup 60 ml rp43200 rp52300...,2.816059,Comtusi obat apa?
3,Copal Balsem 25 g,copal balsem 25 g rp17800 rp25900 oleskan memb...,5.366810,balsem murah?
4,Copal Balsem 36 g,copal balsem 36 g rp19500 rp28700 gosokkan tub...,5.147675,balsem murah?
5,Tresnojoyo Balsem Telon 40 g,tresnojoyo balsem telon 40 g rp16300 rp19700 h...,4.803055,balsem murah?
6,Pamol Sirup 60 ml,pamol sirup 60 ml rp20200 rp44000 makan anak 6...,4.440062,obat pereda nyeri gigi?
7,Pamol 500 mg 10 Tablet,pamol 500 mg 10 tablet rp9800 rp12700 dikonsum...,3.165156,obat pereda nyeri gigi?
8,Neurofenac Plus 50 mg 10 Tablet,neurofenac plu 50 mg 10 tablet rp89500 rp10120...,3.055855,obat pereda nyeri gigi?
9,Boost Optimum 400 g,boost optimum 400 g rp118000 rp216000 larutkan...,5.060993,Mengapa Boost Optimum tidak cocok untuk orang ...


dari hasil yang didapatkan, dapat dilihat bahwa metode bisa cukup baik dalam mencari dokumen yang relevant
- pertanyaan 1 : semua dokumen relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : semua dokumen teratas memang merupakan obat nyeri namun bukan untuk gigi
- pertanyaan 4 : 2 dokumen teratas memang obat dengan nama Boost nmun bukan yang ditanyakan, dokumen ke 3 relevant
- pertanyaan 5 : 2 dokumen teratas tidak relevant, dokumen ke 3 relavant
- pertanyaan 6 : semua dokumen relevant

# Method 3 : Jaccard Similarity

Cara kerja :

terdapat 2 tahap un tuk metode ini

1. menghitung Jaccard Similarity:
    -setiap kata dalam pertanyaan dan dokumen dipecah menjadi set untuk menghilangkan duplikasi.
    - Jccard similarity dihitung sebagai perbandingan elemen yang sama (intersection) dan total elemen yang unik (union).

2. mencari Jaccard Similarity untuk Dokumen:
    - fungsi jaccard_search menghitung kesamaan untuk setiap dokumen terhadap pertanyaan dan mengembalikan skor dalam bentuk array.

3. kemudian diambil dokumen dengan 3 scor terbesar untuk masing-masing pertanyaan

In [7]:
def jaccard_similarity(query, document):
    query_terms = set(query.split())
    doc_terms = set(document.split())
    
    intersection = len(query_terms.intersection(doc_terms))
    union = len(query_terms.union(doc_terms))
    
    return intersection / union if union > 0 else 0

def jaccard_search(query, documents):
    scores = []
    for doc in documents:
        score = jaccard_similarity(query, doc)
        scores.append(score)
    return np.array(scores)

all_results = []
for idx, row in df_eval.iterrows():
    query = row['processed_question']  
    documents = df_obat['processed_text'].tolist() 
    
    jaccard_scores = jaccard_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen' : documents,
        'Jaccard Score': jaccard_scores
    })
    
    results = results.sort_values(by='Jaccard Score', ascending=False).head(3)  
    results['Query'] = row['Pertanyaan']  
    all_results.append(results)

final_results = pd.concat(all_results, ignore_index=True)

final_results

,Nama_Obat,Dokumen,Jaccard Score,Query
0,Konvermex 250 mg 2 Kaplet,konvermex 250 mg 2 kaplet rp6800 rp13500 dosi ...,0.047619,Comtusi obat apa?
1,Provula 50 mg 10 Tablet,provula 50 mg 10 tablet rp203500 rp242900 maka...,0.045455,Comtusi obat apa?
2,Konvermex Suspensi 125 mg 10 ml,konvermex suspensi 125 mg 10 ml rp16600 dosi t...,0.045455,Comtusi obat apa?
3,Balsem Tolak Angin 20 g,balsem tolak angin 20 g rp13400 rp18500 gosokk...,0.021739,balsem murah?
4,Copal Balsem 36 g,copal balsem 36 g rp19500 rp28700 gosokkan tub...,0.020408,balsem murah?
5,Copal Balsem 25 g,copal balsem 25 g rp17800 rp25900 oleskan memb...,0.019608,balsem murah?
6,Pamol Sirup 60 ml,pamol sirup 60 ml rp20200 rp44000 makan anak 6...,0.142857,obat pereda nyeri gigi?
7,Dumin Sirup 60 ml,dumin sirup 60 ml rp38800 rp42700 makan anak 7...,0.088235,obat pereda nyeri gigi?
8,Neurofenac Plus 50 mg 10 Tablet,neurofenac plu 50 mg 10 tablet rp89500 rp10120...,0.074074,obat pereda nyeri gigi?
9,Cussons Baby Cotton Buds Extra Fine 50 Pieces,cusson babi cotton bud extra fine 50 piec rp36...,0.078947,Mengapa Boost Optimum tidak cocok untuk orang ...


Setelah melakukan evaluasi, ditmukan bahwa masih banyak pertanyaan yang tidak mengarah ke dokumen obat yang relevan. Oleh karena itu, perlu dilakukan beberapa perbaikan dalam sistem pencarian.
- pertanyaan 1 : semua dokumen tidak relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : pramol merupakan obat demam anak bukan obat nyeri, Menstrucare memang obat nyeri gigi namun bukan untuk obat nyeri gigi, sedangkan Orinox  merupakan memang obat nyeri gigi setelah oprasi.
- pertanyaan 4 : semua dokumen tidak relevant
- pertanyaan 5 : semua dokumen tidak relevant
- pertanyaan 6 : semua dokumen tidak relevant

# Method 4: LSA

Cara kerja : 
1. dokumen dan Pertanyaan diubah menjadi TF-IDF:
2. menggunakan Singular Value Decomposition (SVD):
    - matriks TF-IDF yang telah dibuat kemudian diolah dengan TruncatedSVD untuk mengurangi dimensi data. Proses ini menangkap struktur laten dalam data, sehingga mengurangi noise dan meningkatkan pemahaman semantik.
3. yransformasi Pertanyaan ke Ruang LSA:
4. menghitung Similarity
5. 3 dokumen dengan nilai similiaritu terbanyak disimpan


In [8]:
def lsa_search(query, documents, n_components=100):
    vectorizer = TfidfVectorizer()
    
    tfidf_matrix = vectorizer.fit_transform(documents)
    
    svd = TruncatedSVD(n_components=n_components)
    lsa_matrix = svd.fit_transform(tfidf_matrix)

    query_vector = vectorizer.transform([query])
    lsa_query = svd.transform(query_vector)

    cosine_scores = cosine_similarity(lsa_query, lsa_matrix).flatten()
    
    return cosine_scores

all_results_lsa = []
for idx, row in df_eval.iterrows():
    query = row['processed_question'] 
    documents = df_obat['processed_text'].tolist()  
    
    lsa_scores = lsa_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen' : documents,
        'LSA Score': lsa_scores
    })
    
    results = results.sort_values(by='LSA Score', ascending=False).head(3)  
    results['Query'] = row['Pertanyaan']  
    all_results_lsa.append(results)

final_results_lsa = pd.concat(all_results_lsa, ignore_index=True)
final_results_lsa

,Nama_Obat,Dokumen,LSA Score,Query
0,Faktu Suppositoria,faktu suppositoria rp10000 rp13100 dimasukan d...,0.714537,Comtusi obat apa?
1,Ultraproct-N Cream 10 g,ultraproctn cream 10 g rp139800 rp146200 olesk...,0.681044,Comtusi obat apa?
2,Ultraproct-N Suppositoria,ultraproctn suppositoria rp21600 rp26100 dimas...,0.677278,Comtusi obat apa?
3,Balsem Otot Geliga 40 g,balsem otot geliga 40 g rp18700 rp23700 dioles...,0.711241,balsem murah?
4,Balsem Otot Geliga 20 g,balsem otot geliga 20 g rp9000 rp17200 diolesk...,0.710152,balsem murah?
5,Balsem Otot Geliga Muscular Balm 10 g,balsem otot geliga muscular balm 10 g rp5700 r...,0.703829,balsem murah?
6,Bufect Suspensi Forte 200mg/5ml 50 ml,bufect suspensi fort 200mg5ml 50 ml rp27900 rp...,0.736609,obat pereda nyeri gigi?
7,Kaltrofen Suppositoria 100 mg,kaltrofen suppositoria 100 mg rp22800 rp29200 ...,0.735485,obat pereda nyeri gigi?
8,Paramex Nyeri Otot 4 Tablet,paramex nyeri otot 4 tablet rp1800 rp2500 dimi...,0.705165,obat pereda nyeri gigi?
9,Boost Optimum 800 g,boost optimum 800 g rp234700 rp410400 larutkan...,0.713155,Mengapa Boost Optimum tidak cocok untuk orang ...


Setelah melakukan evaluasi, ditmukan bahwa masih terdapt beberapa pertanyaan yang tidak mengarah ke dokumen obat yang relevan.

- pertanyaan 1 : semua dokumen tidak relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : semua dokumen relevant
- pertanyaan 4 : semua dokumen relevant
- pertanyaan 5 : semua dokumen relevant
- pertanyaan 6 : semua dokumen relevant, namun 2 dokumne tertinggi memang relevant namun bukan merek yang di inginkan.

# Method 5: BoW

Cara kerja :
1. dukumen diubah menjadi matrik Bag of Words
2. menghitung vektor pertanyaan
3. menghitung Cosine similiarity
4. kemudiaan diambil 3 dokumen dingan nilai kemiripak yang paling tinggi


In [9]:
def bow_search(query, documents):
    vectorizer = CountVectorizer()
    count_matrix = vectorizer.fit_transform(documents)
    query_vector = vectorizer.transform([query])
    cosine_scores = cosine_similarity(query_vector, count_matrix).flatten()
    return cosine_scores

all_results_bow = []
for idx, row in df_eval.iterrows():
    query = row['processed_question'] 
    documents = df_obat['processed_text'].tolist()  
    
    bow_scores = bow_search(query, documents)
    
    results = pd.DataFrame({
        'Nama_Obat': df_obat['name'],
        'Dokumen': documents,
        'BOW Score': bow_scores
    })
    
    results = results.sort_values(by='BOW Score', ascending=False).head(3)
    results['Query'] = row['Pertanyaan']
    all_results_bow.append(results)

final_results_bow = pd.concat(all_results_bow, ignore_index=True)
final_results_bow

,Nama_Obat,Dokumen,BOW Score,Query
0,Ottogenta 0.1% Cream 5 g,ottogenta 01 cream 5 g rp11800 rp21500 oleskan...,0.481156,Comtusi obat apa?
1,Sulfadiazine Silver Cream 35 g,sulfadiazin silver cream 35 g rp25900 dioleska...,0.438529,Comtusi obat apa?
2,Isoniazid 300 mg 10 Tablet,isoniazid 300 mg 10 tablet rp3200 rp6600 kondi...,0.425864,Comtusi obat apa?
3,Copal Balsem 25 g,copal balsem 25 g rp17800 rp25900 oleskan memb...,0.400000,balsem murah?
4,Copal Balsem 36 g,copal balsem 36 g rp19500 rp28700 gosokkan tub...,0.383131,balsem murah?
5,Tresnojoyo Balsem Telon 40 g,tresnojoyo balsem telon 40 g rp16300 rp19700 h...,0.374634,balsem murah?
6,Bufect Suspensi Forte 200mg/5ml 50 ml,bufect suspensi fort 200mg5ml 50 ml rp27900 rp...,0.519701,obat pereda nyeri gigi?
7,Bimastan 500 mg 10 Kaplet,bimastan 500 mg 10 kaplet rp3900 rp6600 makan ...,0.516185,obat pereda nyeri gigi?
8,Paramex Nyeri Otot 4 Tablet,paramex nyeri otot 4 tablet rp1800 rp2500 dimi...,0.513436,obat pereda nyeri gigi?
9,Boost Optimum 800 g,boost optimum 800 g rp234700 rp410400 larutkan...,0.327706,Mengapa Boost Optimum tidak cocok untuk orang ...


dilihat dari hasil yang didapat ternyata metode sudah cukup baik dalam mencari dokumen yang relevant dengan pertanyaan, namun masih tterdapat beberapa pertanyaan yang tidak mendapatkan dokumen obat yng relevan

- pertanyaan 1 : semua dokumen tidak relevan
- pertanyaan 2 : semua dokumen relevant
- pertanyaan 3 : semua dokumen relevan
- pertanyaan 4 : semua dokumen relevant
- pertanyaan 5 : semua dokumen relevant
- pertanyaan 6 : 2 dokumen relevant, namun dokumen ke 3 tidak relevant namun memang berkaitan dengan kulit.

# Evaluate

In [10]:
# Membaca keenam dataset
df1 = pd.read_csv('pertanyaan 1.csv')
df2 = pd.read_csv('pertanyaan 2.csv')
df3 = pd.read_csv('pertanyaan 3.csv')
df4 = pd.read_csv('pertanyaan 4.csv')
df5 = pd.read_csv('pertanyaan 5.csv')
df6 = pd.read_csv('pertanyaan 6.csv')

# Menggabungkan semua DataFrame menjadi satu
all_data = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)


In [11]:
# Fungsi untuk menghitung precision dan recall
def calculate_precision_recall(true_positive, false_positive, false_negative):
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return precision, recall

# Daftar untuk menyimpan hasil evaluasi
results = []

# Loop untuk setiap metode
methods = ['tfidf_search', 'bm25_search', 'jaccard_search', 'lsa_search', 'bow_search']

for method in methods:
    all_results = []  # Simpan hasil untuk semua pertanyaan

    # Loop untuk setiap pertanyaan di df_eval
    for idx, row in df_eval.iterrows():
        query = row['processed_question']
        documents = all_data['processed_text'].tolist()  # Ambil dokumen dari semua data
        
        # Hitung skor dengan metode yang sesuai
        if method == 'tfidf_search':
            scores = tfidf_search(query, documents)
        elif method == 'bm25_search':
            scores = bm25_search(query, documents)
        elif method == 'jaccard_search':
            scores = jaccard_search(query, documents)
        elif method == 'lsa_search':
            scores = lsa_search(query, documents)
        elif method == 'bow_search':
            scores = bow_search(query, documents)

        # Simpan hasil dalam DataFrame sementara
        results_df = pd.DataFrame({
            'Nama_Obat': all_data['name'],
            'Query': query,
            'Score': scores
        })

        # Ambil 3 hasil teratas
        top_results = results_df.sort_values(by='Score', ascending=False).head(3)
        all_results.append(top_results)

    # Gabungkan hasil untuk setiap metode
    final_results = pd.concat(all_results, ignore_index=True)

    # Hitung true positives, false positives, dan false negatives
    # Untuk menghitung ini, kamu perlu membandingkan hasil dengan jawaban yang benar.
    # Misalnya, kita asumsikan df_eval memiliki kolom 'Expected' yang menunjukkan nama obat yang benar.
    true_positive = len(final_results[final_results['Nama_Obat'].isin(df_eval['Expected'])])
    false_positive = len(final_results) - true_positive
    false_negative = len(df_eval['Expected']) - true_positive

    # Hitung precision dan recall
    precision, recall = calculate_precision_recall(true_positive, false_positive, false_negative)

    # Simpan hasil evaluasi untuk setiap metode
    results.append({
        'Method': method,
        'Precision': precision,
        'Recall': recall
    })

# Buat DataFrame hasil evaluasi
evaluation_results = pd.DataFrame(results)

# Tampilkan hasil evaluasi
print(evaluation_results)


KeyError: 'processed_text'